RESTART Prior to Running if any changes made in source files...

### import configuration and functions modules
list_builder also imported for the hybrid list creation example below

In [1]:
%%time
import config as cf
import functions as f
import list_builder as lb

CPU times: user 644 ms, sys: 56 ms, total: 700 ms
Wall time: 1.17 s


### build pkl files
master, fur, sg(special group), active_each_month, order proposals, month percentage, initial editor values

In [2]:
%%time
%run build_program_files

CPU times: user 2.36 s, sys: 96 ms, total: 2.45 s
Wall time: 2.37 s


### build compensation tables
convert from input Excel file, detects config sample_mode if selected

In [3]:
%%time
%run make_pay_tables_from_excel

CPU times: user 244 ms, sys: 0 ns, total: 244 ms
Wall time: 249 ms


### generate the skeleton file

In [4]:
%%time
%run make_skeleton master

CPU times: user 796 ms, sys: 128 ms, total: 924 ms
Wall time: 968 ms


### standalone
standalone must be rerun after job level model change (option is in the config file - enhanced or basic)...

In [5]:
%%time
if cf.compute_with_job_changes:
    %run standalone_with_job_changes skel
else:
    %run standalone_no_job_changes skel

CPU times: user 2.08 s, sys: 312 ms, total: 2.39 s
Wall time: 2.7 s


### Create dataset for each list order
(including list conditions, job count changes, and possible furlough recall schedule).
The p1, p2, and p3 inputs represent different list order proposals - source is the pickle files generated with the build_program_files script.
The "supc", "ratio", and "count" inputs represent conditional job assignment functions to be applied.

In [6]:
%%time
%run compute_measures p1 supc ratio

['supc', 'ratio']
CPU times: user 2 s, sys: 404 ms, total: 2.4 s
Wall time: 2.47 s


In [7]:
%%time
%run compute_measures p2 supc count

['supc', 'count']
CPU times: user 2.08 s, sys: 320 ms, total: 2.4 s
Wall time: 2.49 s


In [8]:
%%time
%run compute_measures p3 supc

['supc']
CPU times: user 1.98 s, sys: 332 ms, total: 2.31 s
Wall time: 2.54 s


### merge an ordered list with the master list

In [9]:
%%time
%run join_inactives master p1 final ffill

CPU times: user 2.19 s, sys: 8 ms, total: 2.2 s
Wall time: 2.2 s


### config file options viewer:

In [10]:
%%time
f.print_config_selections()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.52 ms


,setting
option,
actives_only,False
annual_pcnt_raise,0.02
apply_count_cond,True
apply_ratio_cond,True
apply_supc,True
case_study,sample3
compute_job_category_order,True
compute_pay_measures,True
delayed_implementation,True


## cells below for demonstration only

### build a 'hybrid' list example:
(stored as 'dill/hybrid.pkl')

In [11]:
%%time
prepared_master = lb.prepare_master_list()
lb.build_list(prepared_master, ['ldate', 'jobp'], [.5, .5])

CPU times: user 124 ms, sys: 0 ns, total: 124 ms
Wall time: 124 ms


### samples of files created

In [12]:
%%time
file_names = ['master', 'skel', 'stand', 'active_each_month', 'p1', 'pay_table_enhanced_with_fur_indexed', 'ds1', 'hybrid', cf.case_study + '_final']
for file in file_names:
    df = pd.read_pickle('dill/' + file + '.pkl')
    print('< ' + file.upper() + ' >', '\n', 'row count: ' + str(len(df)),
        '\n', 'column count: '+ str(len(df.columns)), '\n',
        df.columns, '\n',
        '\n', df.head(), '\n\n', '-' * 80, '\n')

< MASTER > 
 row count: 7510 
 column count: 11 
 Index(['empkey', 'eg', 'lname', 'dob', 'retdate', 'doh', 'ldate', 'sg', 'fur',
       'line', 'eg_order'],
      dtype='object') 
 
             empkey  eg    lname        dob    retdate        doh      ldate  \
empkey                                                                        
10011102  10011102   1  tooeyoo 1949-07-13 2014-07-13 1973-02-26 1975-01-29   
10010475  10010475   1  rubelot 1949-02-05 2014-02-05 1975-05-27 1975-05-27   
10013096  10013096   1  yeloxid 1949-01-08 2014-01-08 1977-01-18 1977-01-18   
10012178  10012178   1  xayeaue 1951-06-07 2016-06-07 1977-11-15 1977-11-15   
10014447  10014447   1  finuceu 1951-10-17 2016-10-17 1977-12-09 1977-12-09   

          sg  fur  line  eg_order  
empkey                             
10011102   0    0     1         1  
10010475   0    0     1         2  
10013096   0    0     1         3  
10012178   0    0     1         4  
10014447   0    0     1         5   

 --------